'''
Copyright 2019 Xilinx Inc.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
'''

In [1]:
from ctypes import *
import cv2
import numpy as np
import runner
import os
import input_fn_sl as input_fn
import math
import threading
import time
import sys
from skinl_aux import *

In [4]:
# Transform the dpu_skinl_0.elf into shard library libdpumodelskinl.so
!/usr/bin/aarch64-xilinx-linux-gcc -fPIC -shared \
../coutput/output_zcu102/dpu_skinl_0.elf -o libdpumodelskinl.so

## Lesion detection app

In [3]:

SCRIPT_DIR = get_script_directory()
calib_image_dir  = SCRIPT_DIR + "/../data/calib/"
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
threadnum = 0
runRotal = 0
path = os.getcwd()
batchSize =2



# load lession images
labels = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
print(labels.index('nv'))

org_label_index =[]
img = []
with open(path +'/../data/calibration.txt', 'r') as fp:
    for cnt, line in enumerate(fp):
        org_label_index.append(labels.index(line.split('/')[0] ))
        # append image
        img_path = os.path.join(calib_image_dir,line.rstrip())
        #print(img_path)
        image = cv2.imread(img_path)
        image = cv2.resize(image,(224,224))
        img.append(input_fn.preprocess_fn(image))
        #print("Line {}: {}".format(org_label_index[cnt], line.split('/')[0]))

runTotal = len(img)
runSteps = int(runTotal/batchSize)
print('{} images loaded.'.format(runTotal))

# DPU related
"""create runner """
dpu = runner.Runner(path+'/dpuv2_rundir/' ) 

"""get tensor"""
inputTensors = dpu.get_input_tensors()
outputTensors = dpu.get_output_tensors()
tensorformat = dpu.get_tensor_format() 
if tensorformat == dpu.TensorFormat.NCHW:
    outputHeight = outputTensors[0].dims[2]
    outputWidth = outputTensors[0].dims[3]
    outputChannel = outputTensors[0].dims[1]
elif tensorformat == dpu.TensorFormat.NHWC:
    outputHeight = outputTensors[0].dims[1]
    outputWidth = outputTensors[0].dims[2]
    outputChannel = outputTensors[0].dims[3]
else:
    exit("Format error")
    
outputSize = outputHeight*outputWidth*outputChannel    
softmax = np.empty(outputSize)

softmax =[]
for steps in range(runSteps):
    shapeIn = (batchSize,) + tuple([inputTensors[0].dims[i] for i in range(inputTensors[0].ndims)][1:])
    #print(shapeIn)
    """prepare batch input/output """
    outputData = []
    inputData = []
    outputData.append(np.empty((batchSize,outputSize), dtype = np.float32, order = 'C'))
    inputData.append(np.empty((shapeIn), dtype = np.float32, order = 'C'))

    """init input image to input buffer """
    k = steps * batchSize
    for j in range(batchSize):
        imageRun = inputData[0]
        #print(j)
        imageRun[j,...] = img[j+k].reshape(inputTensors[0].dims[1],inputTensors[0].dims[2],inputTensors[0].dims[3])
    """run with batch """
    job_id = dpu.execute_async(inputData,outputData)
    dpu.wait(job_id)
    #print(inputData[0])

    """softmax calculate with batch """
    for j in range(batchSize):
        softmax.append( CPUCalcSoftmax(outputData[0][j], outputSize))

# delete the dpu resouces
del dpu

# accuracy 
passed = passedp(org_label_index, softmax)
print('Passed: {0}, Failed: {1}, Accuracy: {2:2.2f}%'.format(passed, 600-passed, passed/len(softmax)*100))
print('Top-3 accuracy: {0:2.2f}%'.format(top_n_accuracy(org_label_index,softmax, 3)/len(softmax)*100))


5
600 images loaded.
Passed: 361, Failed: 239, Accuracy: 60.17%
Top-3 accuracy: 83.50%


##    FPS estimation"

In [2]:
SCRIPT_DIR = get_script_directory()
calib_image_dir  = SCRIPT_DIR + "/../data/calib/nv"

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
batchSize = 1
threadnum = 0
runRotal = 0
path = os.getcwd()

"""create runner """
dpu = runner.Runner(path+'/dpuv2_rundir' ) 

#multi threading
l = threading.Lock()
threadAll = []
threadnum = 4 #int(argv[1])

"""image list to be run """
listimage=os.listdir(calib_image_dir)
runTotall = len(listimage)
img = []
for i in range(runTotall):
    path = os.path.join(calib_image_dir,listimage[i])
    image = cv2.imread(path)
    image = cv2.resize(image,(224,224))
    img.append(input_fn.preprocess_fn(image))

    """run with batch """
time1 = time.time()
for i in range(threadnum):
    t1 = threading.Thread(target=skinlv1, args=(dpu, img, i*batchSize, runTotall, batchSize,threadnum, l))
    threadAll.append(t1)
for x in threadAll:
    x.start()
for x in threadAll:
    x.join()

time2 = time.time()

timetotal = time2 - time1
fps = float(runTotall / timetotal)
print("%.2f FPS" %fps)

# Free resouces 
del dpu

cnt : 0 and runTotall: 401cnt : 1 and runTotall: 401
cnt : 2 and runTotall: 401

cnt : 3 and runTotall: 401
139.68 FPS


In [17]:
'''
"""image list to be run """
calib_image_dir  = SCRIPT_DIR + "/../data/images/" 
runTotall = batchSize
listimage=os.listdir(calib_image_dir)
len(listimage)
img = [] 
for i in range(runTotall):
    path = os.path.join(calib_image_dir,listimage[i+1])
    print(path)
    image = cv2.imread(path)
    image = cv2.resize(image,(224,224)) 
    #print(image.shape) 
    imgpi = input_fn.preprocess_fn(image) 
    cv2.imshow(str(i),imgpi)
    #cv2.waitKey(0) 
    img.append(input_fn.preprocess_fn(image))
'''

/home/root/workspace/skinl/target_zcu102/../data/images/ISIC_0025178.jpg


In [5]:
print(img[0].shape)
print(IMAGE_WIDTH)

(224, 224, 3)
224


In [15]:
#print(softmax[0])
for j in range(batchSize):
    print(np.argmax(softmax[j]))

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
4
5
5
5
5
5
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
4
5
5
5
5
5
5
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


IndexError: list index out of range

In [19]:
org_label_index

[5,
 5,
 2,
 5,
 5,
 2,
 5,
 5,
 2,
 5,
 1,
 4,
 5,
 5,
 1,
 5,
 5,
 2,
 5,
 1,
 5,
 5,
 5,
 5,
 0,
 5,
 6,
 5,
 4,
 5,
 4,
 5,
 1,
 2,
 5,
 5,
 5,
 2,
 0,
 5,
 5,
 5,
 5,
 5,
 4,
 5,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 1,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 2,
 5,
 4,
 5,
 5,
 5,
 5,
 5,
 2,
 5,
 5,
 4,
 0,
 5,
 5,
 4,
 4,
 1,
 5,
 1,
 5,
 5,
 5,
 2,
 5,
 5,
 5,
 4,
 4,
 5,
 5,
 5,
 4,
 5,
 5,
 5,
 5,
 5,
 0,
 5,
 1,
 5,
 2,
 5,
 5,
 5,
 2,
 5,
 5,
 5,
 5,
 5,
 0,
 5,
 5,
 4,
 4,
 5,
 4,
 4,
 4,
 4,
 5,
 3,
 2,
 5,
 5,
 5,
 2,
 5,
 5,
 4,
 5,
 2,
 5,
 5,
 5,
 5,
 5,
 0,
 5,
 5,
 5,
 5,
 5,
 1,
 1,
 6,
 5,
 3,
 4,
 4,
 2,
 2,
 5,
 5,
 5,
 5,
 2,
 5,
 5,
 5,
 5,
 2,
 5,
 5,
 0,
 4,
 5,
 5,
 4,
 5,
 5,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 0,
 5,
 2,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 4,
 5,
 5,
 2,
 5,
 5,
 4,
 2,
 1,
 5,
 4,
 5,
 5,
 5,
 5,
 1,
 2,
 5,
 5,
 5,
 5,
 5,
 0,
 5,
 5,
 5,
 5,
 2,
 5,
 5,
 5,
 2,
 5,
 2,
 5,
 5,
 5,
 5,
 1,
 3,
 5,
 4,
 5,
 5,
 1,
 4,
 4,
 2,
 2,


In [21]:
for i in range(600): print(softmax[i], org_label_index[i])

[1.5426936085692847e-13, 4.5987048340228346e-10, 1.3708545905027775e-06, 3.398008800101829e-09, 7.484612460589515e-05, 0.9999237789935931, 1.691768964452731e-10] 5
[4.980016943034879e-07, 2.718997122234187e-05, 0.0014845221281942955, 7.391000469001627e-05, 0.010969217285331567, 0.9874174726376452, 2.718997122234187e-05] 5
[3.0208046517525663e-07, 1.0003531132170638e-05, 0.0014846556575064192, 2.7192416897003783e-05, 0.0109702039409097, 0.9875062885423703, 1.3538307191385862e-06] 2
[0.00011981298009956897, 0.003967660986515035, 0.017781822878071174, 0.0005369645233968703, 0.006541567063394392, 0.9708546333597267, 0.00019753820879613054] 5
[4.2483529233675005e-18, 1.8795282272780295e-12, 5.602794680974823e-09, 7.582558050663146e-10, 3.0590222459676587e-07, 0.999999686484693, 1.2501524744446829e-09] 5
[2.5076795718485484e-08, 1.0116701441372384e-05, 0.00020319938034155091, 7.475287448643311e-05, 0.0009106764419761961, 0.9986779828707469, 0.0001232466542117592] 2
[4.357843012983588e-05, 0.

[1.3520428580009584e-06, 7.381903881237744e-05, 0.0024445508334281292, 0.00020066095179799862, 0.010955716752060652, 0.9862021933615702, 0.00012170701947260504] 4
[5.563439381737023e-09, 3.0375349806438127e-07, 0.0003331061579660351, 5.008048533083155e-07, 0.006690616035167981, 0.9929754621216359, 5.563439381737023e-09] 4
[6.615856649464946e-08, 1.3288303301134484e-06, 0.001457239401927842, 1.3288303301134484e-06, 0.029269435813345582, 0.9692705955348739, 5.430625839672418e-09] 5
[2.543138711786597e-13, 4.59810265675532e-10, 3.725881173529571e-06, 3.3975638479407123e-09, 0.00020342621931803505, 0.9997928440041364, 3.774352502523059e-11] 5
[0.0004809480257691483, 0.005859146419344908, 0.04329356158438817, 0.005859146419344908, 0.07137901586854672, 0.8695744297395278, 0.0035537519430781806] 2
[8.039384408577049e-07, 1.6147535237817358e-05, 0.003951171598646117, 1.6147535237817358e-05, 0.029195428598897656, 0.9668198131802269, 4.876133129017697e-07] 5
[1.1209342151180574e-07, 4.5221713798

[8.422336770586674e-12, 1.522794211005361e-08, 1.0128738282986065e-05, 1.8551431278967023e-07, 0.0001233932929620238, 0.999866208971176, 6.824690171836805e-08] 5
[1.139846879500193e-12, 3.3978356594506905e-09, 3.726179251044255e-06, 4.139411240072095e-08, 0.00012339410997743214, 0.9998728293155996, 5.60208392607975e-09] 5
[1.0379489047761158e-15, 7.570969222091917e-10, 2.7494355303151803e-05, 1.5206698185464463e-08, 0.001501140935906057, 0.9984713335382964, 1.5206698185464463e-08] 4
[3.098567879667411e-12, 9.23670066928552e-09, 6.143714167110521e-06, 6.825049941438099e-08, 7.48457607371123e-05, 0.9999189178060522, 1.522874486454115e-08] 5
[2.2892302486124468e-11, 2.5104457979058333e-08, 1.669802017688442e-05, 1.8549824834050921e-07, 0.00020342352996059916, 0.9997796264340102, 4.1390253859471024e-08] 5
[2.1151307938742386e-19, 1.6918977276654978e-10, 1.1253516175234742e-07, 1.1548222842366915e-17, 4.599054848724273e-10, 0.9999998847745895, 2.0611533849412856e-09] 5
[7.562877383220862e-1

[3.419343588381246e-14, 1.6805283322452776e-10, 2.7351403818830672e-05, 2.047302625836918e-09, 0.006692667850568654, 0.9932799785271449, 3.0779950075691206e-12] 5
[3.231641196065772e-12, 3.190139522997689e-07, 0.2326939990172652, 1.0564291211535559e-05, 0.3836475457440398, 0.3836475457440398, 2.6186259835531047e-08] 5
[1.388733410451679e-11, 3.0588888963419187e-07, 1.6700967489719506e-05, 6.825303693170259e-08, 1.6700967489719506e-05, 0.9999560942603773, 1.0129648829378816e-05] 5
[6.003135253641166e-06, 0.00019879653709348622, 0.003992935185993064, 0.0005403850143418081, 0.017895093981633407, 0.9770390260664371, 0.0003277600792475578] 1
[8.039437965628008e-07, 1.614764281002981e-05, 0.00395119792069864, 9.794040446371343e-06, 0.029195623094020386, 0.9668262539741199, 1.7938410807739441e-07] 2
[5.97684577828742e-06, 0.0039754489614243944, 0.01781672616010252, 3.6251402129053368e-06, 0.001462485942334397, 0.9727602879887232, 0.0039754489614243944] 4
[7.577759622278334e-10, 5.040283409979

207

In [ ]:
del dpu